# Validation Set Split

As of Dec 07, 2018

## Part A: 분석을 위한 환경 설정

### A-1) 모듈/패키지 로드

In [94]:
evaluation = True
evaluation_verbose = False

OUTPUT_BUCKET_FOLDER = "gs://line_2018/output/"
DATA_BUCKET_FOLDER = "gs://upload-bigquery180927/data/"

In [95]:
from IPython.display import display

In [96]:
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.ml.linalg import Vectors, SparseVector, VectorUDT

In [97]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all" # 한 셀(cell)에서의 코드 실행 결과가 다 보이도록 설정

In [98]:
import numpy as np
import scipy.sparse

In [99]:
import warnings 
warnings.filterwarnings('ignore') # warning 메시지 반환되지 않도록 설정

In [100]:
import math
import datetime
import time
import itertools

In [101]:
import pickle

In [102]:
import random
random.seed(42)

In [103]:
import pandas as pd
%matplotlib inline

In [104]:
# 테이블 로드
train_valid_merged_df = spark.read.parquet("gs://line_2018/outputtrain_final_3")

## Part B: 테이블 합치기

In [105]:
# CTR에 해당하는 컬럼 제외
train_valid_merged_df = train_valid_merged_df.drop('pop_ad_id',
 'pop_ad_doc_id',
 'pop_ad_publisher_id',
 'pop_advertiser_id',
 'pop_campaign_id',
 'pop_view_doc_ad_doc',
 'pop_ad_source_id',
 'pop_ad_topic_id',
 'pop_ad_category_id')

In [106]:
# train_valid_merged_df.columns

In [107]:
# column별 수준 수 세기
# for col in train_valid_merged_df.columns:
#   print(col, train_valid_merged_df.select(col).distinct().count())

In [108]:
# column별 NA 세기
# train_valid_merged_df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in train_valid_merged_df.columns]).show()

In [109]:
train_valid_merged_df2 = train_valid_merged_df

In [110]:
# 상당수를 차지하는 미국, 캐나다, 영국이 아닌 경우 U로 할당
train_valid_merged_df2 = train_valid_merged_df2.withColumn("country", \
              F.when((train_valid_merged_df2["country"] != 'US') &
                     (train_valid_merged_df2["country"] != 'CA') &
                     (train_valid_merged_df2["country"] != 'GB'), 'U').otherwise(train_valid_merged_df2["country"]))

In [111]:
# train_valid_merged_df2.select('country').distinct().count()

In [112]:
# train_valid_merged_df2.select('country').groupBy('country').count().show()

In [113]:
cont_binned_schema = StructType(
                    [StructField("pop_ad_id", StringType(), True),
                    StructField("pop_ad_doc_id", StringType(), True),                    
                    StructField("pop_ad_publisher_id", StringType(), True),
                    StructField("pop_advertiser_id", StringType(), True),
                    StructField("pop_campaign_id", StringType(), True),
                    StructField("pop_view_doc_ad_doc", StringType(), True),
                     StructField("pop_ad_source_id", StringType(), True),
                     StructField("pop_ad_topic_id", StringType(), True),
                     StructField("pop_ad_category_id", StringType(), True),
                    ]
                    )

cont_binned_df = spark.read.schema(cont_binned_schema).options(header='true', inferschema='false', nullValue='\\N') \
                .csv(DATA_BUCKET_FOLDER + "train_bin.csv")

In [114]:
# # column별 수준 수 세기 
# for col in cont_binned_df.columns:
#   print(col, cont_binned_df.select(col).distinct().count())

In [115]:
# cont_binned_df.count()

In [116]:
# train_valid_merged_df2.count()

In [117]:
# column별 NA 세기
# cont_binned_df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in cont_binned_df.columns]).show()

In [118]:
# cont_binned_df.select('pop_ad_id').groupBy('pop_ad_id').count().show()

In [119]:
# NA인 값을 U로 바꾸기
for col in cont_binned_df.columns:
  cont_binned_df = cont_binned_df.withColumn(col, F.regexp_replace(col, 'NA', 'U'))

In [120]:
# cont_binned_df.select('pop_ad_id').groupBy('pop_ad_id').count().show()

In [121]:
# cont_binned_df.count()

In [122]:
# train_valid_merged_df2.count()

In [123]:
from pyspark.sql.window import Window

cont_binned_df = cont_binned_df.withColumn('index', F.row_number().over(Window.orderBy(F.lit(1))))

In [124]:
# cont_binned_df.select('index').describe().show()

In [125]:
train_valid_merged_df2 = train_valid_merged_df2.withColumn('index', F.row_number().over(Window.orderBy(F.lit(1))))

In [126]:
# train_valid_merged_df2.select('index').describe().show()

In [127]:
train_final = train_valid_merged_df2.join(cont_binned_df, 'index', how = 'left')

In [128]:
# train_final.columns

In [129]:
# train_final.count() # 87141731 기원!

In [130]:
# train_final.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in train_final.columns]).show()

## Part C: Train / Test Set으로 구분

실험을 위해 전체 데이터의 일부를 떼내어 `train_sample`과 `test_sample` 를 생성해보자.

In [131]:
train_final = train_final.drop('is_train') # is_train 컬럼 제외

In [132]:
# 주말인지 여부
# train_final.select('weekend').groupBy('weekend').count().show()

In [133]:
dp_id_table = train_final.select('display_id').distinct()

In [153]:
dp_id_list = dp_id_table.sample(False, 0.015, seed = 0)

In [154]:
dp_id_list.count() # 랜덤 샘플링을 통해 추출된 고유한 display의 수: 약 252,531건 기대

252531

In [155]:
dp_id_list = dp_id_list.select('display_id').toPandas()
dp_id_list = dp_id_list['display_id'].tolist() # display_id를 list로 변환

In [156]:
dp_id_all = train_final.filter(F.col('display_id').isin(dp_id_list)) # 랜덤 샘플링 해서 뽑힌 display_id 기준으로 train_final을 필터링

In [157]:
dp_id_list = dp_id_all.select('display_id', 'timestamp_event').distinct()

In [158]:
dp_id_list = dp_id_list.sort("timestamp_event").withColumn('index', F.row_number().over(Window.orderBy(F.lit(1))))

In [160]:
dp_id_list_train = dp_id_list.filter((F.col('index') >= 1) & (F.col('index') <= 200000)).select('display_id').toPandas()
dp_id_list_train = dp_id_list_train['display_id'].tolist()

In [161]:
dp_id_list_test = dp_id_list.filter((F.col('index') >= 200001) & (F.col('index') <= 252531)).select('display_id').toPandas()
dp_id_list_test = dp_id_list_test['display_id'].tolist()

In [162]:
train_sample = train_final.filter(F.col('display_id').isin(dp_id_list_train)) # dp_id_list_train에 있는 display_id 기준으로 train_final을 필터링

In [163]:
test_sample = train_final.filter(F.col('display_id').isin(dp_id_list_test)) # dp_id_list_test에 있는 display_id 기준으로 train_final을 필터링

In [164]:
train_sample.count() # train_sample의 행 개수: 1,032,825

1032825

In [165]:
test_sample.count() # test_sample의 행 개수: 272,177

272177

이제 `index` 컬럼을 제거하자.

In [166]:
train_sample = train_sample.drop('index')

In [167]:
test_sample = test_sample.drop('index')

csv와 파케이 형식으로 파일을 생성하였다.

In [168]:
# csv 파일로 쓰기: test_sample_df_2
test_sample.repartition(1).write.csv(OUTPUT_BUCKET_FOLDER + 'test_sample_df_2.csv', header = True)

In [169]:
# csv 파일로 쓰기: train_sample_df_2
train_sample.repartition(1).write.csv(OUTPUT_BUCKET_FOLDER + 'train_sample_df_2.csv', header = True)

In [170]:
# 파케이로 쓰기: test_sample_df_2
test_sample.write.parquet(OUTPUT_BUCKET_FOLDER + 'test_sample_parquet_2', mode='overwrite')

In [171]:
# 파케이로 쓰기: train_sample_df_2
train_sample.write.parquet(OUTPUT_BUCKET_FOLDER + 'train_sample_parquet_2', mode='overwrite')

## Part D: 최종 Train Set 생성

분석에 사용할 Train, Test 셋을 만들어보자.

In [ ]:
# dp_id_table = train_final.select('display_id').distinct()

In [ ]:
# train/test 셋 구분
# dp_train_id_list = dp_id_table.sample(False, 0.7, seed = 0)

In [ ]:
# dp_train_id_list = dp_train_id_list.select('display_id').toPandas()
# dp_train_id_list = dp_train_id_list['display_id'].tolist() # display_id를 list로 변환

In [ ]:
# dp_train_id_all = train_final.filter(F.col('display_id').isin(dp_id_list)) # 랜덤 샘플링 해서 뽑힌 display_id 기준으로 train_final을 필터링

In [ ]:
# index 컬럼 없애기
# train_final = train_final.drop('index')

In [ ]:
# 파케이로 쓰기: train_final
# train_final.write.parquet(OUTPUT_BUCKET_FOLDER + 'train_final_parquet', mode='overwrite')